In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Conv2D, Flatten, MaxPooling2D
from keras.layers import Dense, Dropout

Using TensorFlow backend.
C:\Users\hp\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\hp\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\hp\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\hp\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarn

In [2]:
train = pd.read_csv('train_ctrUa4K.csv')

In [3]:
test = pd.read_csv('test_lAUu6dG.csv')

In [4]:
submission = pd.read_csv('sample_submission_49d68Cx.csv')

In [5]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [6]:
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [7]:
submission.head()

,Loan_ID,Loan_Status
0,LP001015,N
1,LP001022,N
2,LP001031,N
3,LP001035,N
4,LP001051,N


In [8]:
X_train = train.drop("Loan_Status", axis = 'columns')
y_train = train.Loan_Status
X_test = test

In [9]:
train.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [10]:
X_train = X_train.fillna(0)
X_test = X_test.fillna(0)

In [11]:
X_test.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
dtype: int64

In [12]:
X_train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001002,Male,No,0,Graduate,No,5849,0.0,0.0,360.0,1.0,Urban
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban


In [13]:
dependents = pd.get_dummies(X_train['Dependents'], drop_first=True)
gender = pd.get_dummies(X_train['Gender'], drop_first=True)
married = pd.get_dummies(X_train['Married'], drop_first=True)
education = pd.get_dummies(X_train['Education'], drop_first=True)
self_employed = pd.get_dummies(X_train['Self_Employed'], drop_first=True)
property_area = pd.get_dummies(X_train['Property_Area'], drop_first=True)


X_train = pd.concat([X_train, dependents, gender, married, education, self_employed, property_area], axis=1)
X_train = X_train.drop(['Dependents','Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area'], axis=1)


In [14]:
dependents = pd.get_dummies(X_test['Dependents'], drop_first=True)
gender = pd.get_dummies(X_test['Gender'], drop_first=True)
married = pd.get_dummies(X_test['Married'], drop_first=True)
education = pd.get_dummies(X_test['Education'], drop_first=True)
self_employed = pd.get_dummies(X_test['Self_Employed'], drop_first=True)
property_area = pd.get_dummies(X_test['Property_Area'], drop_first=True)


X_test = pd.concat([X_test, dependents, gender, married, education, self_employed, property_area], axis=1)
X_test = X_test.drop(['Dependents','Gender', 'Married', 'Education', 'Self_Employed', 'Property_Area'], axis=1)


In [15]:
X_train.head()

,Loan_ID,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,0,1,2,3+,Female,Male,No,Yes,Not Graduate,No,Yes,Semiurban,Urban
0,LP001002,5849,0.0,0.0,360.0,1.0,1,0,0,0,0,1,1,0,0,1,0,0,1
1,LP001003,4583,1508.0,128.0,360.0,1.0,0,1,0,0,0,1,0,1,0,1,0,0,0
2,LP001005,3000,0.0,66.0,360.0,1.0,1,0,0,0,0,1,0,1,0,0,1,0,1
3,LP001006,2583,2358.0,120.0,360.0,1.0,1,0,0,0,0,1,0,1,1,1,0,0,1
4,LP001008,6000,0.0,141.0,360.0,1.0,1,0,0,0,0,1,1,0,0,1,0,0,1


In [16]:
X_train = X_train.drop('Loan_ID', axis = 'columns')
X_test = X_test.drop('Loan_ID', axis = 'columns')
X_train = X_train.drop('No', axis = 'columns')
X_test = X_test.drop('No', axis = 'columns')

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [18]:
model = RandomForestClassifier()

In [19]:
model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [20]:
y_pred = model.predict(X_test)

In [22]:
Loan_Status = submission.Loan_ID
output = pd.DataFrame(Loan_Status)
output['Loan_Status'] = y_pred

In [23]:
output

,Loan_ID,Loan_Status
0,LP001015,Y
1,LP001022,Y
2,LP001031,Y
3,LP001035,Y
4,LP001051,N
...,...,...
362,LP002971,Y
363,LP002975,Y
364,LP002980,N
365,LP002986,Y


In [24]:
output.to_csv("sample_submission.csv", index= False)